In [13]:
from datasets import load_dataset
import json
with open ("/ssddata/ksshumab/Cont_Pretrain_Math/own_LMFlow/LMFlow/data/alpaca/train/train_52002.json") as f:
    json_data = json.load(f)
data_files=["/ssddata/ksshumab/Cont_Pretrain_Math/own_LMFlow/LMFlow/data/alpaca/test/test_252.json"]
KEY_INSTANCES = "instances"

extensions = "json"
raw_dataset = load_dataset(
    extensions,
    data_files=data_files,
    field=KEY_INSTANCES,
    split="train",
    use_auth_token=None,
)

raw_dataset

/home/data/ksshumab/anaconda3/envs/slmflow/lib/python3.10/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(


Dataset({
    features: ['input', 'output'],
    num_rows: 252
})

In [14]:
raw_dataset[0]

{'input': 'Instruction: The sentence you are given might be too wordy, complicated, or unclear. Rewrite the sentence and make your writing clearer by keeping it concise. Whenever possible, break complex sentences into multiple sentences and eliminate unnecessary words. If you have any questions about my rate or if you find it necessary to increase or decrease the scope for this project, please let me know.',
 'output': "If you have any questions about my rate or find it necessary to increase or decrease this project's scope, please let me know."}

In [9]:
column_names = list(raw_dataset.features)
column_names
tokenized_column_order = ["input", "output"]
label_columns = ["output"]


In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
def tokenize_function(examples):
    
    num_example = len(examples[column_names[0]])
    token_dict = {
        "input_ids": [[] for _ in range(num_example)],
        "attention_mask": [[] for _ in range(num_example)],
        "labels": [[] for _ in range(num_example)],
    }
    print('1')

    for column_name in tokenized_column_order:
        encoding = tokenizer(
            examples[column_name],
            add_special_tokens=True,
            truncation=True,
        )
        print('2')
        if column_name in label_columns:
            labels = encoding["input_ids"].copy()
        else:
            labels = [
                [-100] * len(encoding["input_ids"][i])
                for i in range(num_example)
            ]

        for i in range(num_example):
            token_dict["input_ids"][i].extend(
                encoding["input_ids"][i]
            )
            token_dict["attention_mask"][i].extend(
                encoding["attention_mask"][i]
            )
            token_dict["labels"][i].extend(labels[i])

    for i in range(num_example):
        block_size = 1024
        max_length = 1024
        pad_length = max_length - len(token_dict["input_ids"][i])

        if pad_length < 0:
            # Truncates too long samples
            for key in ["input_ids", "attention_mask", "labels"]:
                token_dict[key][i] = token_dict[key][i][:pad_length]
        else:
            # Pads too short samples
            pad_token_id = tokenizer.pad_token_id
            token_dict["input_ids"][i].extend(
                [pad_token_id for _ in range(pad_length)]
            )
            token_dict["attention_mask"][i].extend(
                [0 for _ in range(pad_length)]
            )
            token_dict["labels"][i].extend(
                [-100 for _ in range(pad_length)]
            )
    return token_dict

In [15]:
tokenized_datasets = raw_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset: 100%|██████████| 252/252 [00:00<00:00, 2482.10 examples/s]


In [22]:
tokenized_datasets[0]

{'input_ids': [6310,
  2762,
  25,
  383,
  6827,
  345,
  389,
  1813,
  1244,
  307,
  1165,
  1573,
  88,
  11,
  8253,
  11,
  393,
  10061,
  13,
  16140,
  6525,
  262,
  6827,
  290,
  787,
  534,
  3597,
  22363,
  416,
  5291,
  340,
  35327,
  13,
  21326,
  1744,
  11,
  2270,
  3716,
  13439,
  656,
  3294,
  13439,
  290,
  11005,
  13114,
  2456,
  13,
  1002,
  345,
  423,
  597,
  2683,
  546,
  616,
  2494,
  393,
  611,
  345,
  1064,
  340,
  3306,
  284,
  2620,
  393,
  10070,
  262,
  8354,
  329,
  428,
  1628,
  11,
  3387,
  1309,
  502,
  760,
  13,
  1532,
  345,
  423,
  597,
  2683,
  546,
  616,
  2494,
  393,
  1064,
  340,
  3306,
  284,
  2620,
  393,
  10070,
  428,
  1628,
  338,
  8354,
  11,
  3387,
  1309,
  502,
  760,
  13,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  